In [ ]:
import jupyter_aws_sso
import time
import importlib
importlib.reload(jupyter_aws_sso)

jupyter_aws_sso.login("Jupyter-IR-AdministratorAccess")


# Organization Configuration
This notebook will configure the organization to have the resources needed to run the Jupyter Incident Response runbooks across the organization. This **must be run in the organizational managemebnt account.**.

The Athena tables can be set up in a delagate account so no resources will be deployed into the management account.

## Accounts:
* **Management Account**: The root account that owns the AWS organization. Typically resources are note deployed to the root account, but some configurations must be done in this account. For example, the cloudtrail organizational trail is created in the management account.
* **Logging**: The account that holds the log files. Typically this account contains the buckets for the CloudTrail and other logs, and Athena tables used to query them.
* **CloudFormation StackSet Delegated Account**: An account designated by the management account to own stacksets.

## Parameters
The following cell holds the parameters for the environment

* **athena_database**: The name of the athena database that will be used to query logs in S3.
* **athena_cloudtrail_table**: The name of the athena table that will be used to query CloutTrail logs in S3.
* **flow_logs_bucket**: The bucket that holds the organizational flow logs. Leave empty and one will be created automatically.
* **enable_flow_logs_automatically**: Automatically enable flow logs in all VPCs in the organization. See [blog](https://aws.amazon.com/blogs/mt/how-to-enable-vpc-flow-logs-automatically-using-aws-config-rules/)


In [ ]:
# Save Settings


external_cloud_trail = False # Set this to True if the organization cloudtrail is managed externally.
athena_database = "jupyter-incident-response"  # <--- To customize, replace the value with the athena database name
athena_cloudtrail_table = "cloudtrail_logs"  # <--- To customize, replace the value with the athena table name for cloudtrail logs
flow_logs_bucket = ""  # <--- Leave blank and a bucket will be created automatically
flowlog_output_format = "parquet"
cloudtrail_bucket = ""  # <--- Leave blank and a bucket will be created automatically
enable_flow_logs_automatically = True
opt_in_out = "in"
region_list = "us-east-1,us-west-2,us-east-2"
aggregator_id = ""  # <--- Leave blank and an aggregator will be created automatically


## CloudFormation
This notebook uses cloudformation to deploy the cloudformation stacks and stack sets used to configure the organization. 

The templates have feature toggles to turn on an off the creation of the configuration. If the resources have already been created outside of this notebook, the resource creation will be skipped.
* **logging-account.yaml**: 
  * If necessary. creates the S3 buckets for organization-wide logging
  * Creates Athena tables to query logs
* **account-level-resources.yaml**:
  * Note: Deployed as a stackset from the **Delegated Stack Stack** account to every account under the root.
  * Creates global resources in each member account.
  * Creates a stackset to deployed to each region in the account
    * Configures Flow Log Auto Remediation
    * 
* **managage-account.yaml**: If necessary, configures organizational CloudTrail logs.

# Organizational Onboarding

The notebook will validate and update the organization to configure the logging and configuration of the organization to the other runbooks have access to the data and services needed to execute the runbooks.

This runbook is designed to be able to be run multiple times, so it will not create additional resources if the configurate has already been found.

The following services will be configured.
* **CloudTrail**: An organization trail will be configured to store the CloudTrail logs in a bucket in the management account. The first trail in each account is included free of charge. If an organzation trail already exists, a new one will not be created.
* **VPC Flow Logs**: Any VPCs with the tag 'runbooks-enable-flowlogs' set to 'true' will have VPC flow logs enabled into a central bucket.
* **Athena**: Athena will be configured to query the following service logs:
  * CloudTrail
  * VPC Flow Logs
  * **Up Next**: AWS Config
  * ***Future*** Application Load Balancer
  * ***Future*** Elastic Load Balancer
  * ***Future*** CloudFront
  * ***Future*** Amazon EMR
  * ***Future*** Global Accelerator
  * ***Future*** GuardDuty
  * ***Future*** Network Firewall
  * ***Future*** Network Load Balancer
  * ***Future*** Route 53
  * ***Future*** Amazon SES
  * ***Future*** AWS WAF
  

## Load Settings

The information on how to connect to data in the AWS environment is managed through SSM Parameter Store.

The athena database and tablenames can be provided below. The runbook will manage the Athena tables so they are configured with the columns and partitioning expected by the notebooks.

In [ ]:
# Load Settings
import boto3
ssm_client = boto3.client('ssm')

put_response = ssm_client.put_parameter(Name='Jupyter-Athena-Database', Value=athena_database, Type="String", Overwrite=True)
sso_portal_url_response = ssm_client.put_parameter(Name='Jupyter-Athena-Cloudtrail', Value=athena_cloudtrail_table, Type="String", Overwrite=True)

get_management_account_response = ssm_client.get_parameter(Name='Jupyter-Management-Account')
get_cfn_delegate_account_response = ssm_client.get_parameter(Name='Jupyter-CfnDelegateAccount')
get_logging_account_response = ssm_client.get_parameter(Name='Jupyter-LoggingAccount')

management_account = get_management_account_response['Parameter']['Value']
cfn_delegate_account = get_cfn_delegate_account_response['Parameter']['Value']
logging_account = get_logging_account_response['Parameter']['Value']

ec2_client = boto3.client('ec2')

stack_set_region_list = ""

if opt_in_out == "out":
    region_opt_out = region_list.split(',')
    stack_set_regions = []

    region_response = ec2_client.describe_regions()

    for region in region_response['Regions']:
        if not region['RegionName'] in region_opt_out:
            stack_set_regions.append(region['RegionName'])
    stack_set_region_list =','.join(stack_set_regions)
else:
    stack_set_region_list = region_list

management_account

In [ ]:
# Configure Logging Account
org_client = boto3.client('organizations')
desc_client = org_client.describe_organization()

org_id = desc_client['Organization']['Id']

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"

!aws cloudformation deploy --template-file cfn-templates/logging-account.yaml --stack-name organization-logging --parameter-overrides RegionList=$stack_set_region_list FlowLogsFormat=$flowlog_output_format DatabaseName=$athena_database CloudTrailLogsBucket=$cloudtrail_bucket FlowLogsBucket=$flow_logs_bucket OrgId=$org_id CloudTrailTableName=$athena_cloudtrail_table --capabilities CAPABILITY_IAM --profile $profile

**Note** Organizational Rules don't support remediation, so the rule will be deployed in each region.


In [ ]:
config_bucket = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='ConfigBucket'].OutputValue" --output text
config_bucket = config_bucket[0]

config_topic = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='ConfigTopic'].OutputValue" --output text
config_topic = config_topic[0]


cfn_bucket = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='CfnBucket'].OutputValue" --output text
cfn_bucket = cfn_bucket[0]


cfn_bucket = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='CfnBucket'].OutputValue" --output text
cfn_bucket = cfn_bucket[0]

cfn_cloudtrail_bucket = !aws cloudformation describe-stacks --stack-name organization-logging --profile $profile  --query "Stacks[0].Outputs[?OutputKey=='CloudTrailLogsBucket'].OutputValue" --output text
cfn_flowlog_bucket = !aws cloudformation describe-stacks --stack-name organization-logging --profile $profile  --query "Stacks[0].Outputs[?OutputKey=='BucketName'].OutputValue" --output text

cfn_cloudtrail_bucket = cfn_cloudtrail_bucket[0]
cfn_flowlog_bucket = cfn_flowlog_bucket[0]

print(f"config_bucket={config_bucket}\nconfig_topic={config_topic}\ncfn_bucket={cfn_bucket}\ncfn_cloudtrail_bucket={cfn_cloudtrail_bucket}\ncfn_flowlog_bucket={cfn_flowlog_bucket}")


## Validate CloudTrail
The next section will check the CloudTrail logs and validate an organizational trail is configured.

Then the cloudformation stack will be created in the management account to configure the organization trail if one is needed.

In [ ]:
flowlog_output_format

In [ ]:


org_trail_info = {}
org_trail_arn = ""
found_org_trail = True

cloudtrail_client = boto3.client('cloudtrail', region_name = "us-east-1")

if external_cloud_trail == True:
    found_org_trail = False

    paginator = cloudtrail_client.get_paginator('list_trails')

    response_iterator = paginator.paginate()
    for page in response_iterator:
        for trail in page["Trails"]:
            trail_response = cloudtrail_client.get_trail(Name=trail['TrailARN'])
            if trail_response['Trail']['IsOrganizationTrail'] and trail_response['Trail']['IsMultiRegionTrail']:
                found_org_trail = True
                org_trail_info = trail_response['Trail']
                org_trail_arn = org_trail_info['TrailARN']

    if not found_org_trail:
        raise ("External cloud trail not found, please configure.")

!aws cloudformation deploy --template-file cfn-templates/managage-account.yaml --stack-name management-organization --parameter-overrides LoggingAccount=$logging_account CloudTrailArn=$org_trail_arn TrailBucket=$cfn_cloudtrail_bucket --capabilities CAPABILITY_IAM


In [ ]:
trail_arn = !aws cloudformation describe-stacks --stack-name management-organization --query "Stacks[0].Outputs[?OutputKey=='TrailArn'].OutputValue" --output text

trail_response = cloudtrail_client.get_trail(Name=trail_arn[0])
org_trail_info = trail_response['Trail']

print(f"Organization Trail was found: {org_trail_info['Name']} in region {org_trail_info['HomeRegion']}")

### Create Trail, if needed
If an organization trail was not found, the following section will create one using a cloudformation template.

## Account and Region Configurations

Any configurations that need to deployed to each account will be handled with this stackset.


# Configure member accounts for flow logs and AWS Config
The next section will create a CloudFormation stack that configures the AWS Config and flowlogs in every account and every region.



In [ ]:

org_unit_id = org_client.list_roots()['Roots'][0]['Id']


In [ ]:
import random, string

config_bucket = '' # Disable config recorder 

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"
random_str=''.join(random.choices(string.ascii_uppercase + string.digits, k=8))

!aws s3 sync cfn-templates/ s3://$cfn_bucket/$random_str/ --profile $profile > /dev/null

profile = f"Jupyter-IR-AdministratorAccess-{cfn_delegate_account}"
template_url = f'https://s3.us-east-1.amazonaws.com/{cfn_bucket}/{random_str}'

!aws cloudformation deploy --stack-name jupyter-ir-config --template-file cfn-templates/organization-ir-config.yaml --parameter-overrides ConfigTopic=$config_topic ConfigBucket=$config_bucket OrgUnitId=$org_unit_id OutputFormat=$flowlog_output_format  AutomateRemediationTemplateUrl=$template_url RegionList=$stack_set_region_list --profile $profile --capabilities CAPABILITY_NAMED_IAM



In [ ]:

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"
!aws s3 rm --recursive s3://$cfn_bucket/$random_str --profile $profile > /dev/null

## Test CloudTrail access with Athena
The following code uses a helper library to query cloudtrail logs and return them as a dataframe

In [ ]:
import jupyter_athena
import importlib
importlib.reload(jupyter_athena)

accountid = "383086473915"

sql = f"""
SELECT * 
FROM \"{athena_database}\".\"{athena_cloudtrail_table}\"
WHERE accountId = '{accountid}'
LIMIT 10
"""

display(jupyter_athena.run_query(sql))


## Flow Logs
Enable athena to query organizational flow logs.

This needs to be deployed in two steps. 
1. Deploy CFN The central bucket and athena query in the management account
1. The stacksets for all regions/accounts in organization.

In [ ]:


display(jupyter_athena.get_vpc_flow_by_account("913149361159"))


In [ ]:
# Delete Flow Logs
import boto3

# delete this in the future or move to another notebook

if False: # Don't do this by default, it exists to delete all the flow logs in an org if the deployment needs to be redone.

    accounts = ["913149361159","251344881676", "959309154643", "669399753706" ]



    for account in accounts:
        profile = f"Jupyter-IR-AdministratorAccess-{account}"
        !aws s3 rm --recursive s3://$cfn_flowlog_bucket/AWSLogs/$account --profile Jupyter-IR-AdministratorAccess-251344881676
        for region_name in stack_set_region_list.split(','):
            session = boto3.session.Session(profile_name=profile, region_name=region_name)
            client = session.client('ec2')
            response = client.describe_flow_logs()

            flow_log_ids = []
            for flow_log in response['FlowLogs']:
                flow_log_id = flow_log['FlowLogId']
                flow_log_ids.append(flow_log_id)

            if len(flow_log_ids) > 0:
                delete_response = client.delete_flow_logs(FlowLogIds=flow_log_ids)
                print(f"deleted: {flow_log_ids} in region {region_name} in account {account}")

In [ ]:


for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in stack_set_region_list.split(','):
        session = boto3.session.Session(profile_name=profile, region_name=region_name)
        client = session.client('ec2')
        response = client.describe_flow_logs()

        flow_log_ids = []
        for flow_log in response['FlowLogs']:
            flow_log_id = flow_log['FlowLogId']
            print(f"FlowLogId: {flow_log_id}:{flow_log['DestinationOptions']['FileFormat']} in region {region_name} in account {account}")

## AWS Config

It is a best practice to enable AWS config across the organization to collect data on the current state of the resoruces deployed in the organization, establish detective controls for compliance, and make resources available through a query interface.

In [ ]:
# Get all the accounts in the organization
org_client = boto3.client('organizations')
paginator = org_client.get_paginator('list_accounts')

accounts = []

page_iterator = paginator.paginate()
for page in page_iterator:
    for account in page['Accounts']:
        accounts.append(account['Id'])
        
accounts

### Check delivery channels

Verify a delivery channel exists for the every account and every region im the organizational.


In [ ]:

accounts = []

paginator = org_client.get_paginator('list_accounts')
page_iterator = paginator.paginate()
for page in page_iterator:        
    for acct in page['Accounts']:
        accounts.append(acct['Id'])

for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in stack_set_region_list.split(','):
        
        session = boto3.session.Session(profile_name=profile, region_name=region_name)
        config_client = session.client('config')
        delivery_channels_result = config_client.describe_delivery_channels()
        delivery_channel_found = False
        for delivery_channel in delivery_channels_result['DeliveryChannels']:
            delivery_channel_found = True
        
        if delivery_channel_found:
            print(f"Delivery Channel found in account {account} in region {region_name}")
        else:
            print(f"Delivery Channel NOT found in account {account} in region {region_name}")

## Set up the multi-account multi-region aggregator

An aggregator is an AWS Config resource type that collects AWS Config configuration and compliance data from the following:

* Multiple accounts and multiple regions.
* Single account and multiple regions.
* An organization in AWS Organizations and all the accounts in that organization which have AWS Config enabled.
* Use an aggregator to view the resource configuration and compliance data recorded in AWS Config.

In [ ]:
profile = f"Jupyter-IR-AdministratorAccess-{management_account}"
!aws cloudformation deploy --stack-name aws-config-aggregator --template-file cfn-templates/aws-config-aggregator.yaml --parameter-overrides AggregatorId=$aggregator_id RegionList=$stack_set_region_list --profile $profile --capabilities CAPABILITY_IAM

print('It can take 10 minutes for the aggregator to be ready to field a query')

## Test AWS Config Aggregator Query
The following query will display the first 100 the running EC2 instances in the organization. Remember, it may take a few minutes for the aggregator to completely deployed across the organization.

The **jupyter_awsconfig** is a helper library to simplify calling aws config queries and returning a pandas dataframe. 

In [ ]:
import json
import jupyter_awsconfig

import importlib
importlib.reload(jupyter_awsconfig)

query_expression = f"""
SELECT
  accountId,
  awsRegion,
  resourceId,
  configuration.instanceType,
  availabilityZone,
  configuration.state.name
WHERE
  resourceType = 'AWS::EC2::Instance'
"""

df = jupyter_awsconfig.run_query(query_expression, 100)
display(df)